In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from thalamus_merfish_analysis import ccf_plots as cplots
from thalamus_merfish_analysis import ccf_images as cimg
from thalamus_merfish_analysis import abc_load as abc
get_ipython().run_line_magic('matplotlib', 'inline') 

# Load brain3 data

In [3]:
# load all cells in the brain - realigned just loads faster version here
obs = abc.get_combined_metadata(realigned=True, version="20230830", drop_unused=False)
obs.columns

## load CCF structures

In [4]:
realigned=False
ccf_images = abc.get_ccf_labels_image(resampled=True, realigned=realigned)
if realigned:
    ccf_label = 'parcellation_substructure_realigned'
    coords = 'section'
else:
    ccf_label = 'parcellation_substructure'
    coords = 'reconstructed'

## Preprocessing

### Subset to thalamus & neurons

In [5]:
# get thalamus CCF substructure names
th_names = abc.get_thalamus_substructure_names()
th_subregion_names = list(set(th_names).difference(['TH-unassigned']))

In [6]:
# remove non-neuronal and some other outlier non-thalamus types
obs_neurons = abc.filter_by_class_thalamus(obs, filter_midbrain=False)

In [7]:
# filter to thalamus boundaries (add a buffer here if wanted)
filter_buffer = 0  # 5
obs_th_neurons = abc.filter_by_thalamus_coords(obs_neurons, 
                                               realigned=realigned, 
                                               buffer=filter_buffer)
obs_th_strict = obs_th_neurons

### Set TH sections

In [8]:
section_col = 'z_section'
sections_all = sorted(obs_th_neurons[section_col].unique())

In [9]:
# 3 sections that representatively span the AP axis of the thalamus
sections_GRC = [6.4, 7.2, 8.0] # 7.8 - dropped due to layout contraints on the poster

### Precalculate CCF boundaries

In [10]:
sections_int = np.rint(np.array(sections_all)/0.2).astype(int)
ccf_boundaries = cimg.sectionwise_label_erosion(ccf_images, distance_px=1, 
                                                fill_val=0, return_edges=True, 
                                                section_list=sections_int
                                                )
# can skip and def as none to calculate on the fly
# ccf_boundaries = None

# View CCF Structures

## View CCF as in ARA

I want to be able to generate the view you get from the ARA, but subset to just the thalamus.


TODO
1. Confirm hex colors from ABC atlas metadata (used Illustratoyr eyedropper of ARA screenshot)
2. Add Allen Reference Atlas labels to plot on top of the centroid(?) of the nucleus
3. Use different salmon shades for the appropriate nuclei, as done in the actual ARA images

In [12]:
ara_figs = cplots.plot_ccf_overlay(obs_th_neurons.iloc[[],:], # no foreground cells; just the ccf shapes
                                    ccf_images, sections=sections_GRC, point_hue=ccf_label, 
                                    # highlight='all', # not currently implemented
                                    legend='ccf',
                                    face_palette=None,
                                    edge_color='black',
                                    section_col=section_col,
                                    x_col = 'x_'+coords,
                                    y_col = 'y_'+coords,
                                    min_section_count=0, 
                                    boundary_img=ccf_boundaries);
# save as pdfs
for i, fig in enumerate(ara_figs):
    sec = int(sections_GRC[i]*10)
    fig.savefig(f'/results/ccf_ara_z{sec}.pdf', transparent=True, 
                bbox_inches='tight')

## view taxonomy labels over CCF

# View Cell Types + CCF Overlay

## Select thalamus celltype subsets

In [13]:
subclasses_all = obs_th_strict['subclass'].value_counts().loc[lambda x: x>100].index
print(f'{len(subclasses_all) = }')

supertypes_all = obs_th_strict['supertype'].value_counts().loc[lambda x: x>20].index
print(f'{len(supertypes_all) = }')

In [14]:
supertypes_from_subclasses = obs_th_neurons.loc[
    lambda df: df['subclass'].isin(subclasses_all), 'supertype'].unique()

print(f'{supertypes_from_subclasses.shape[0] = }')

print(f'{len(supertypes_all.intersection(supertypes_from_subclasses)) = }')

In [15]:
clusters_from_subclasses = obs_th_neurons.loc[
            lambda df: df['subclass'].isin(subclasses_all), 'cluster'].unique()
print(f'{clusters_from_subclasses.shape[0] = }')

### Figure parameters

#### Set celltype color palettes

In [16]:
abc_palettes = {level: abc.get_taxonomy_palette(level) for level in 
                ['subclass','supertype','cluster']}

In [17]:
palette_df = pd.read_csv('/code/resources/cluster_palette_glasbey.csv')
cluster_palette = dict(zip(palette_df['Unnamed: 0'], palette_df['0']))

#### Set plot kwargs

In [18]:
kwargs = dict(
    bg_cells=obs_th_strict,
    section_col=section_col,
    x_col = 'x_'+coords,
    y_col = 'y_'+coords,
    s=3, 
    face_palette=None,
    edge_color='grey',
    boundary_img=ccf_boundaries
)

#### Get obs with TH subclasses

In [19]:
obs_th_subclasses = obs_th_neurons.loc[lambda df: df['subclass'].isin(subclasses_all)]

## Subclass + CCF plots

In [20]:
subclass_figs = cplots.plot_ccf_overlay(obs_th_subclasses, ccf_images, 
                                        point_hue='subclass', 
                                        sections=sections_GRC,
                                        point_palette=abc_palettes['subclass'],
                                        legend=None, #'cells', 
                                        **kwargs)

# save as pdfs
for i, fig in enumerate(subclass_figs):
    sec = int(sections_GRC[i]*10)
    fig.savefig(f'/results/ccf_subclass_z{sec}.pdf', transparent=True, 
                bbox_inches='tight')

In [21]:
subclass_figs = cplots.plot_ccf_overlay(obs_th_subclasses, ccf_images, 
                                        point_hue='subclass', 
                                        sections=sections_GRC,
                                        point_palette=abc_palettes['subclass'],
                                        legend='cells', 
                                        **kwargs)

# save as pdfs
for i, fig in enumerate(subclass_figs):
    sec = int(sections_GRC[i]*10)
    fig.savefig(f'/results/ccf_subclass_z{sec}.pdf', transparent=True, 
                bbox_inches='tight')

## Supertype + CCF plots - NOT DISPLAYING FOR GRC

In [22]:
# # Not displaying supertypes on the GRC poster for simplicity
# obs = obs_th_neurons.loc[lambda df: df['supertype'].isin(supertypes_all)]

# cplots.plot_ccf_overlay(obs_th_subclasses, ccf_images, 
#                         point_hue='supertype', sections=sections_GRC,
#                         point_palette=palettes['supertype'],
#                         legend=None, #'cells', 
#                         **kwargs);

# # save as pdfs
# for i, fig in enumerate(subclass_figs):
#     sec = int(sections_GRC[i]*10)
#     fig.savefig(f'/results/ccf_supertype_z{sec}.pdf', transparent=True, 
#                 bbox_inches='tight')

## Cluster + CCF plots

In [23]:
cluster_figs = cplots.plot_ccf_overlay(obs_th_subclasses, ccf_images, 
                                       sections=sections_GRC,
                                       point_hue='cluster', 
                                       point_palette=cluster_palette,
                                       legend=None, 
                                       **kwargs)

# save as
for i, fig in enumerate(cluster_figs):
    sec_name = int(sections_GRC[i]*10)
    fig.savefig(f'/results/ccf_cluster_z{sec_name}.pdf', transparent=True,  
                bbox_inches='tight')

# Specific nuclei examples

## AD & AV clusters

In [24]:
ant_sections = [8.0]
ad_av_nuclei = ['AD', 'AV']
ad_av_clusters = [str(x) for x in list(range(2613, 2616+1))]
ad_av_obs = obs_th_neurons.loc[lambda df: df['cluster'].str[:4].isin(ad_av_clusters)]

kwargs = dict(
    bg_cells=obs_th_strict,
    section_col=section_col,
    x_col = 'x_'+coords,
    y_col = 'y_'+coords,
    s=4, 
    face_palette=None,
    edge_color='grey',
    boundary_img=ccf_boundaries
)

figs = cplots.plot_ccf_overlay(ad_av_obs, ccf_images, sections=ant_sections,
                               point_hue='cluster', point_palette=cluster_palette,
                               ccf_names=['AD', 'AV'], legend='cells',
                               custom_xy_lims=[3.7,7.6, 5.9, 4.6],
                               **kwargs);
figs[0].savefig(f'/results/abc_ccf_AD_AV_clusters_z{int(ant_sections[0]*10)}.pdf', 
               bbox_inches='tight')

## MD

In [25]:
nucleus='MD'
md_sections = [6.8, 7.0, 7.2]
md_xy_lims = [4.3, 6.8, 6.2, 4.4]
md_kwargs = dict(
                bg_cells=obs_th_strict,
                section_col=section_col,
                x_col = 'x_'+coords,
                y_col = 'y_'+coords,
                s=10, 
                face_palette=None,
                edge_color='grey',
                boundary_img=ccf_boundaries
            )

In [26]:
taxonomy_level = 'subclass'
md_figs = cplots.plot_ccf_overlay(obs_th_subclasses, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=abc_palettes[taxonomy_level],
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend=None, 
                                  **md_kwargs);

for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_{taxonomy_level}_z{sec}.pdf', transparent=True,
                bbox_inches='tight')

In [27]:
# taxonomy_level = 'supertype'
# md_figs = cplots.plot_ccf_overlay(obs_th_subclasses, 
#                                   ccf_images, 
#                                   sections=md_sections,
#                                   point_hue=taxonomy_level, 
#                                   point_palette=abc_palettes[taxonomy_level],
#                                   ccf_names=[nucleus],
#                                   custom_xy_lims=md_xy_lims,
#                                   legend=None, 
#                                   **md_kwargs);

# for i, fig in enumerate(md_figs):
#     sec = int(md_sections[i]*10)
#     fig.savefig(f'/results/ccf_MD_{taxonomy_level}_z{sec}.pdf', transparent=True,
#                 bbox_inches='tight')

In [28]:
# clusters with >=1% of cells in the MD
# see metrics notebook for code
md_clusters = ['2678 TH Prkcd Grin2c Glut_10',
                 '2648 TH Prkcd Grin2c Glut_1',
                 '2635 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2681 TH Prkcd Grin2c Glut_12',
                 '2644 CM-IAD-CL-PCN Sema5b Glut_4',
                 '2682 TH Prkcd Grin2c Glut_12',
                 '2683 TH Prkcd Grin2c Glut_13',
                 '2638 CM-IAD-CL-PCN Sema5b Glut_2',
                 '2677 TH Prkcd Grin2c Glut_10',
                 '2636 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2664 TH Prkcd Grin2c Glut_6',
                 '2687 TH Prkcd Grin2c Glut_13',
                 '2636 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2664 TH Prkcd Grin2c Glut_6',
                 '2687 TH Prkcd Grin2c Glut_13',
                 '2639 CM-IAD-CL-PCN Sema5b Glut_2',
                 '2633 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2674 TH Prkcd Grin2c Glut_9',
                 '2634 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2643 CM-IAD-CL-PCN Sema5b Glut_4',
                 '2662 TH Prkcd Grin2c Glut_6',
                 '2622 PVT-PT Ntrk1 Glut_2',
                 '2621 PVT-PT Ntrk1 Glut_2',
                 '2642 CM-IAD-CL-PCN Sema5b Glut_3',
                 '2637 CM-IAD-CL-PCN Sema5b Glut_2',
                 '2663 TH Prkcd Grin2c Glut_6',
                 '2632 PVT-PT Ntrk1 Glut_6'           
                   ]
md_clusters_gc = ['2678 TH Prkcd Grin2c Glut_10',
                 '2648 TH Prkcd Grin2c Glut_1',
                 '2635 CM-IAD-CL-PCN Sema5b Glut_1',
                 '2683 TH Prkcd Grin2c Glut_13']
md_clusters_1pct = md_clusters[:13]
md_clusters_2pct = md_clusters[:8]
md_clusters_top4 = md_clusters[:4]
md_clusters_5pct = md_clusters[:3]

obs_md = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters)]
obs_md_grc = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters_gc)]
obs_md_1pct = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters_1pct)]
obs_md_2pct = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters_2pct)]
obs_md_top4 = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters_top4)]
obs_md_5pct = obs_th_neurons.loc[lambda df: df['cluster'].isin(md_clusters_5pct)]
obs_md_annot = abc.get_obs_from_annotated_clusters(nucleus, obs_th_neurons)

In [29]:
md_clusters_top4

In [30]:
taxonomy_level = 'cluster'
md_figs = cplots.plot_ccf_overlay(obs_md, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);

for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_{taxonomy_level}_z{sec}.pdf', transparent=True,
                bbox_inches='tight')

In [31]:
taxonomy_level = 'cluster'
md_figs = cplots.plot_ccf_overlay(obs_md_5pct, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);

In [32]:
taxonomy_level = 'cluster'
# plt.rcParams.update({'font.size': 14})
md_figs = cplots.plot_ccf_overlay(obs_md_top4, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);



for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_top4clusters_z{sec}.pdf', transparent=True,
                bbox_inches='tight')
    obs_md_grc

In [33]:
taxonomy_level = 'cluster'
# plt.rcParams.update({'font.size': 14})
md_figs = cplots.plot_ccf_overlay(obs_md_grc, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);

for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_4clusters_z{sec}.pdf', transparent=True,
                bbox_inches='tight')

In [34]:
taxonomy_level = 'cluster'
# plt.rcParams.update({'font.size': 14})
md_figs = cplots.plot_ccf_overlay(obs_md_top4, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus],
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);



for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_top4clusters_z{sec}.pdf', transparent=True,
                bbox_inches='tight')

In [35]:

taxonomy_level = 'cluster'
md_figs = cplots.plot_ccf_overlay(obs_md_2pct, 
                                  ccf_images, 
                                  sections=md_sections,
                                  point_hue=taxonomy_level, 
                                  point_palette=cluster_palette,
                                  ccf_names=[nucleus], 
                                #   highlight=[nucleus], # not currently implemented
                                  custom_xy_lims=md_xy_lims,
                                  legend='cells', 
                                  **md_kwargs);

for i, fig in enumerate(md_figs):
    sec = int(md_sections[i]*10)
    fig.savefig(f'/results/ccf_MD_8clusters_z{sec}.pdf', transparent=True,
                bbox_inches='tight')

In [36]:
# taxonomy_level = 'cluster'
# md_figs = cplots.plot_ccf_overlay(obs_md_1pct, 
#                                   ccf_images, 
#                                   sections=md_sections,
#                                   point_hue=taxonomy_level, 
#                                   point_palette=cluster_palette,
#                                   ccf_names=[nucleus],
#                                   custom_xy_lims=md_xy_lims,
#                                   legend='cells', 
#                                   **md_kwargs);

In [37]:
md_clust_old = ['1130 TH Prkcd Grin2c Glut_1', '1133 TH Prkcd Grin2c Glut_10',
                '1136 TH Prkcd Grin2c Glut_12', '1137 TH Prkcd Grin2c Glut_12',
                '1138 TH Prkcd Grin2c Glut_13', '1142 TH Prkcd Grin2c Glut_13',
                '1157 TH Prkcd Grin2c Glut_6', '1158 TH Prkcd Grin2c Glut_6',
                '1159 TH Prkcd Grin2c Glut_6', '1168 TH Prkcd Grin2c Glut_8']
md_clust_curr = abc.convert_taxonomy_labels(md_clust_old, 'cluster', 
                                        label_format='id_label',
                                        input_version='20230630', output_version='20230830',
                                        output_as_dict=True)
md_clust_curr

In [38]:
abc.convert_taxonomy_labels(['2635 CM-IAD-CL-PCN Sema5b Glut_1'], 'cluster', 
                            label_format='id_label',
                            input_version='20230830', output_version='20230630',
                            output_as_dict=True)

## Clusters, no CCF overlay

In [39]:
# obs = obs_th_neurons.loc[lambda df: df['subclass'].isin(subclasses_all)]
# nuclei = ['AD', 'AV', 'AMd', 'AMv']
# kwargs = dict(
#     bg_cells=obs_th_strict,
#     section_col=section_col,
#     x_col = 'x_'+coords,
#     y_col = 'y_'+coords,
#     s=5, 
#     face_palette=None,
#     edge_color='grey',
#     boundary_img=ccf_boundaries
# )
# cplots.plot_ccf_overlay(obs, ccf_images, 
#                         point_hue='cluster', sections=[sections_GRC[1]],
#                         point_palette=palettes['cluster'],
#                         ccf_names=nuclei,
#                         legend=None, 
#                         **kwargs);

## Gene expression, no CCF overlay

In [40]:
# adata = abc.load_adata_thalamus(transform='log2', subset_to_TH_ZI=True, with_metadata=True,
#                        flip_y=False, round_z=True, 
#                        with_colors=True, realigned=False)

# adata = abc.filter_by_class_thalamus(adata, filter_nonneuronal=True, 
#                                   filter_midbrain=True, 
#                                   filter_other_nonTH=True)

In [41]:
# cplots.plot_expression_ccf(adata, 'Gad2', ccf_images, 
#                            sections=[7.2], 
#                            nuclei=['AD'], ccf_highlight=['AD'], # not in this section so result is no ccf overlays
#                            s=0.5, cmap='Blues', show_outline=False, 
#                            axes=False,  
#                            section_col='z_reconstructed', 
#                            x_col='x_reconstructed',
#                            y_col='y_reconstructed',
#                            boundary_img=None)

In [42]:
# cplots.plot_expression_ccf(adata, 'Slc17a7', ccf_images, 
#                            sections=[7.2], 
#                            nuclei=['AD'], ccf_highlight=['AD'], # not in this section so result is no ccf overlays
#                            s=0.5, cmap='Blues', show_outline=False, 
#                            axes=False,  
#                            section_col='z_reconstructed', 
#                            x_col='x_reconstructed',
#                            y_col='y_reconstructed',
#                            boundary_img=None)